# Loading the Officers who are companies

We have a data file of officers who have a level of control over the company. We need to map these into the system to connect them to the PSC companies we are aware of.

**NOTE** This notebook is incomplete as we don't have sufficient data in the datafile to connect companies together due to missing Companies House IDs.

In [33]:
import pandas as pd
import json
from pandas.io.json import json_normalize

import blaze as bz

First load some sample data and do some tidying up so we don't waste RAM.

In [34]:
master_path = './data/'

We will loop over the officers file and extract out the Companies that are acting as officers and which are still active to reduce our file sizes.

In [19]:
num=0
chunks = pd.read_csv(master_path+'officers.csv', chunksize=500000)
company_officers = []
for chunk in chunks:
    num += 1
    valid_entries = chunk[((chunk.end_date.isnull()) & (chunk.type == 'Company'))]
    print(num, valid_entries.shape[0])
    company_officers.append(valid_entries)

1 11470
2 14593
3 12482
4 16320
5 14182


/home/ec2-user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


6 33316
7 12145
8 11783
9 11773
10 31399
11 12929
12 12739
13 14525
14 29614
15 13348


/home/ec2-user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,4,5,13,15,16,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


16 118710
17 13511
18 12872
19 12429
20 17657
21 12066
22 15353
23 18421
24 13845
25 13857
26 13774
27 14226
28 14125
29 12881
30 12322
31 14155
32 13086
33 12276
34 13259
35 11924
36 12108
37 11942
38 13104
39 13163
40 15661
41 15916
42 14095
43 15827
44 18972
45 21039
46 23076
47 18185
48 18271
49 20743
50 22339
51 33230
52 23721
53 20230
54 19970
55 25887
56 20423
57 20914
58 21210
59 20086
60 18646
61 20841
62 19045
63 17857
64 19777
65 21458
66 18675
67 16616
68 14064
69 14653
70 12824
71 15212
72 13147
73 11936
74 13614
75 13428
76 11432
77 13433
78 11731
79 12762
80 10669
81 7403


In [20]:
company_officers_df = pd.concat(company_officers)

In [35]:
company_officers_df.head()

,id,company_number,jurisdiction_code,name,title,first_name,last_name,position,start_date,person_number,...,partial_date_of_birth,type,address.in_full,address.street_address,address.locality,address.region,address.postal_code,address.country,retrieved_at,source_url
5,190421841,04066366,gb,FATBRAIN.COM LIMITED,NaN,NaN,FATBRAIN.COM LIMITED,secretary,2000-09-06,8.343593e+10,...,NaN,Company,PREMIER SUITE 4 CHURCHILL COURT\n58 STATION RO...,PREMIER SUITE 4 CHURCHILL COURT\n58 STATION ROAD,NORTH HARROW,MIDDLESEX,HA2 7ST,UNITED KINGDOM,2015-12-04 00:00:00 UTC,https://beta.companieshouse.gov.uk/company/040...
7,190421865,04066366,gb,A COMPANY LIMITED,NaN,NaN,+A COMPANY LIMITED,director,2000-09-06,7.227961e+10,...,NaN,Company,PREMIER SUITE 4 CHURCHILL COURT\n58 STATION RO...,PREMIER SUITE 4 CHURCHILL COURT\n58 STATION ROAD,NORTH HARROW,MIDDLESEX,HA2 7ST,NaN,2015-12-04 00:00:00 UTC,https://beta.companieshouse.gov.uk/company/040...
63,204782582,06872285,gb,NOMINEE SECRETARY LTD,NaN,NaN,NOMINEE SECRETARY LTD,secretary,2009-04-07,1.375379e+11,...,NaN,Company,"SUITE B 29 HARLEY STREET, LONDON, W1G 9QR, UNI...",SUITE B 29 HARLEY STREET,LONDON,NaN,W1G 9QR,UNITED KINGDOM,2015-12-04 00:00:00 UTC,https://beta.companieshouse.gov.uk/company/068...
66,204782648,06872285,gb,NOMINEE DIRECTOR LTD,NaN,NaN,NOMINEE DIRECTOR LTD,director,2009-04-07,1.375379e+11,...,NaN,Company,"SUITE B 29 HARLEY STREET, LONDON, W1G 9QR, UNI...",SUITE B 29 HARLEY STREET,LONDON,NaN,W1G 9QR,UNITED KINGDOM,2015-12-04 00:00:00 UTC,https://beta.companieshouse.gov.uk/company/068...
73,201052282,06363382,gb,L4 YOU CO SEC LTD,NaN,NaN,L4 YOU CO SEC LTD,secretary,2007-09-06,9.342384e+10,...,NaN,Company,"5 JUPITER HOUSE CALLEVA PARK\nALDERMASTON, REA...",5 JUPITER HOUSE CALLEVA PARK\nALDERMASTON,READING,BERKSHIRE,RG7 8NN,NaN,2015-12-04 00:00:00 UTC,https://beta.companieshouse.gov.uk/company/063...


## PROBLEM!

**The data appears incomplete** as the id column does not appear to correlate with the actual Companies House registration id for the company in question. This data needs to be re-enriched in order for these connections to be put into the neo4j database!